In [1]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from mlxtend.frequent_patterns import apriori,association_rules

df_train = pd.read_csv("/kaggle/input/network-intrusion-detection/Train_data.csv")

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   urgent                       25192 non-null  int64  
 9   hot                          25192 non-null  int64  
 10  num_failed_logins            25192 non-null  int64  
 11  logged_in                    25192 non-null  int64  
 12  num_compromised              25192 non-null  int64  
 13  root_shell      

In [2]:
df_train.head(n=10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
5,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
6,0,tcp,private,S0,0,0,0,0,0,0,...,9,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
7,0,tcp,private,S0,0,0,0,0,0,0,...,15,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
8,0,tcp,remote_job,S0,0,0,0,0,0,0,...,23,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
9,0,tcp,private,S0,0,0,0,0,0,0,...,13,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [3]:
data_train = pd.get_dummies(df_train,columns = ["protocol_type", "flag", "service", "class"],)

data_train.head()


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois,class_anomaly,class_normal
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# extract numerical attributes and scale it to have zero mean and unit variance  
cols = data_train.select_dtypes(include=['float64','int64']).columns
sc_train = scaler.fit_transform(data_train.select_dtypes(include=['float64','int64']))


# turn the result back to a dataframe
sc_traindf = pd.DataFrame(sc_train, columns = cols)
sc_traindf.head(n=10)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,-0.113551,-0.009889,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,-0.328634,-0.813985,-0.779157,-0.280673,0.073120,-0.287993,-0.641804,-0.627365,-0.221668,-0.374281
1,-0.113551,-0.010032,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,0.732059,-1.030895,-1.157831,2.764403,2.375620,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281
2,-0.113551,-0.010093,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,0.732059,-0.804947,-0.935081,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281
3,-0.113551,-0.009996,0.052473,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,1.238197,-0.021873,...,-1.540854,1.264742,1.069663,-0.440940,-0.380894,0.073759,-0.574435,-0.604947,-0.385140,-0.342768
4,-0.113551,-0.010010,-0.034582,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,1.238197,-0.021873,...,0.732059,1.264742,1.069663,-0.440940,-0.478183,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281
5,-0.113551,-0.010093,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,0.732059,-0.868212,-1.001906,-0.066984,-0.478183,-0.287993,-0.641804,-0.627365,2.884296,2.777041
6,-0.113551,-0.010093,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,0.732059,-0.958592,-1.068731,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281
7,-0.113551,-0.010093,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,0.732059,-0.904364,-1.024181,-0.066984,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281
8,-0.113551,-0.010093,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,0.732059,-0.832060,-0.957356,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281
9,-0.113551,-0.010093,-0.039310,-0.00891,-0.091223,-0.006301,-0.091933,-0.02622,-0.807626,-0.021873,...,0.732059,-0.922440,-1.046456,-0.120406,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281


In [5]:
def encode_units(x):
    if x<=0:
        return 0
    if x>=0 :
        return 1
    
train_df = sc_traindf.applymap(encode_units)

train_df.head(n=10)

    

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,1,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,1,1,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
8,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   duration                     25192 non-null  int64
 1   src_bytes                    25192 non-null  int64
 2   dst_bytes                    25192 non-null  int64
 3   land                         25192 non-null  int64
 4   wrong_fragment               25192 non-null  int64
 5   urgent                       25192 non-null  int64
 6   hot                          25192 non-null  int64
 7   num_failed_logins            25192 non-null  int64
 8   logged_in                    25192 non-null  int64
 9   num_compromised              25192 non-null  int64
 10  root_shell                   25192 non-null  int64
 11  su_attempted                 25192 non-null  int64
 12  num_root                     25192 non-null  int64
 13  num_file_creations           25192 non-null  i

In [7]:
columns = data_train.columns
colname = ['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate']
for col in columns :
    for j in colname :
        if col == j :
            data_train[col] = train_df[col]
data_train.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois,class_anomaly,class_normal
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
data_train[['num_outbound_cmds','is_host_login']].head()
for col in data_train.columns :
    if data_train[col].mean() == 0 :
        print(col)

num_outbound_cmds
is_host_login


In [9]:
data_train.drop(["num_outbound_cmds", "is_host_login"], axis = 1, inplace=True)
data_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Columns: 118 entries, duration to class_normal
dtypes: int64(36), uint8(82)
memory usage: 8.9 MB


In [10]:
frequent_itemsets = apriori ( data_train , min_support = 0.01 , use_colnames=True ,max_len =2)
result_desc = frequent_itemsets.sort_values(['support'],ascending =[False])
result_desc


,support,itemsets
26,0.814782,(protocol_type_tcp)
15,0.643022,(dst_host_count)
12,0.622102,(same_srv_rate)
32,0.594355,(flag_SF)
227,0.562837,"(same_srv_rate, flag_SF)"
...,...,...
281,0.010241,"(flag_RSTO, dst_host_count)"
100,0.010241,"(num_compromised, flag_SF)"
287,0.010162,"(service_finger, dst_host_count)"
47,0.010043,"(duration, srv_rerror_rate)"


In [11]:
rules = association_rules(result_desc , metric = "confidence" , min_threshold = 0.90)
rules = rules.sort_values(['confidence','lift'], ascending=[False , False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
104,(service_eco_i),(protocol_type_icmp),0.036083,0.065695,0.036083,1.000000,15.221752,0.033712,inf
111,(service_ecr_i),(protocol_type_icmp),0.024333,0.065695,0.024333,1.000000,15.221752,0.022735,inf
91,(service_domain_u),(protocol_type_udp),0.072245,0.119522,0.072245,1.000000,8.366656,0.063610,inf
118,(flag_RSTR),(rerror_rate),0.019728,0.124127,0.019728,1.000000,8.056284,0.017280,inf
131,(flag_RSTO),(rerror_rate),0.012067,0.124127,0.012067,1.000000,8.056284,0.010569,inf
...,...,...,...,...,...,...,...,...,...
122,(hot),(flag_SF),0.020641,0.594355,0.018736,0.907692,1.527188,0.006468,4.394484
0,(same_srv_rate),(flag_SF),0.622102,0.594355,0.562837,0.904735,1.522212,0.193088,4.258046
84,(dst_bytes),(service_http),0.098206,0.317680,0.088719,0.903395,2.843725,0.057521,7.063009
109,(service_other),(dst_host_count),0.034058,0.643022,0.030764,0.903263,1.404717,0.008863,3.690211


In [12]:
rules[rules['consequents'] == {'class_anomaly'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
45,(flag_S0),(class_anomaly),0.278223,0.46614,0.275048,0.988586,2.120792,0.145357,46.772805
44,(dst_host_srv_serror_rate),(class_anomaly),0.281478,0.46614,0.277072,0.984346,2.111697,0.145864,34.104513
36,(srv_serror_rate),(class_anomaly),0.286440,0.46614,0.277707,0.969512,2.079873,0.144186,17.510607
43,(dst_host_serror_rate),(class_anomaly),0.287512,0.46614,0.277191,0.964103,2.068270,0.143170,14.872106
31,(serror_rate),(class_anomaly),0.288544,0.46614,0.278184,0.964094,2.068250,0.143682,14.868307
56,(service_private),(class_anomaly),0.172714,0.46614,0.164814,0.954263,2.047160,0.084306,11.672486
123,(flag_RSTR),(class_anomaly),0.019728,0.46614,0.018617,0.943662,2.024417,0.009421,9.476014
113,(service_ecr_i),(class_anomaly),0.024333,0.46614,0.022825,0.938010,2.012292,0.011482,8.612004


In [13]:
frequent_itemsets = apriori ( data_train , min_support = 0.33 , use_colnames=True ,max_len =2)
result_desc = frequent_itemsets.sort_values(['support'],ascending =[False])
result_desc


,support,itemsets
6,0.814782,(protocol_type_tcp)
3,0.643022,(dst_host_count)
2,0.622102,(same_srv_rate)
7,0.594355,(flag_SF)
19,0.562837,"(same_srv_rate, flag_SF)"
9,0.533860,(class_normal)
21,0.516077,"(dst_host_count, protocol_type_tcp)"
20,0.509963,"(same_srv_rate, class_normal)"
33,0.502660,"(flag_SF, class_normal)"
5,0.498730,(dst_host_same_srv_rate)


In [14]:
rules = association_rules(result_desc , metric = "confidence" , min_threshold = 0.75)
rules = rules.sort_values(['confidence','lift'], ascending=[False , False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
21,(logged_in),(protocol_type_tcp),0.394768,0.814782,0.394768,1.000000,1.227321,0.073118,inf
13,(dst_host_srv_count),(same_srv_rate),0.435773,0.622102,0.430891,0.988796,1.589443,0.159795,33.728142
8,(dst_host_same_srv_rate),(same_srv_rate),0.498730,0.622102,0.492339,0.987186,1.586854,0.182078,29.490107
23,(logged_in),(flag_SF),0.394768,0.594355,0.387861,0.982504,1.653058,0.153229,23.184690
24,(logged_in),(same_srv_rate),0.394768,0.622102,0.387702,0.982102,1.578682,0.142116,21.113444
14,(dst_host_srv_count),(dst_host_same_srv_rate),0.435773,0.498730,0.426485,0.978685,1.962355,0.209152,23.516858
26,(count),(dst_host_count),0.364481,0.643022,0.351818,0.965258,1.501129,0.117449,10.275159
25,(logged_in),(class_normal),0.394768,0.533860,0.378533,0.958874,1.796115,0.167782,11.334383
4,(class_normal),(same_srv_rate),0.533860,0.622102,0.509963,0.955238,1.535500,0.177848,8.442437
1,(flag_SF),(same_srv_rate),0.594355,0.622102,0.562837,0.946971,1.522212,0.193088,7.126276


In [15]:
rules[rules['consequents'] == {'class_normal'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,(logged_in),(class_normal),0.394768,0.53386,0.378533,0.958874,1.796115,0.167782,11.334383
18,(dst_host_srv_count),(class_normal),0.435773,0.53386,0.401437,0.921206,1.725557,0.168795,5.915937
11,(dst_host_same_srv_rate),(class_normal),0.498730,0.53386,0.432637,0.867479,1.624918,0.166386,3.517468
5,(flag_SF),(class_normal),0.594355,0.53386,0.502660,0.845722,1.584165,0.185357,3.021435
3,(same_srv_rate),(class_normal),0.622102,0.53386,0.509963,0.819742,1.535500,0.177848,2.585963
